In [9]:
import numpy as np
import pandas as pd
from typing import List


In [17]:
# # Super basic function to create an 'Exercise' line

# def exerciseLine(name: str, sets: int, reps: int, weight) -> str:
#     return f'{name}: {sets}x{reps}, {weight}lbs'

# def exerciseLineFromReps(name: str, reps: List[int], weight) -> str:
#     if len(set(reps)) == 1:
#         return f'{name}: {len(reps)}x{reps}, {weight}lbs'
#     else:
#         repStr = '/'.join([str(rep) for rep in reps])
#         return f'{name}: {repStr}, {weight}lbs'


# For List[Weight] and List[List[Reps]], generate a line
# s["Reps list"] = [[a, b], [c, d]]
# s["Weight"] = [x, y]
def generateLineFromSeries(s):
    name, repsList, weights = s["Exercise Name"], s["Reps"], s["Weight"]
    if name == "Intensity":
        return f"Intensity: {repsList[0][0]}"
    
    blocks = []
    for weight, reps in zip(weights, repsList):
        # Round to the nearest .5 (for floating point errors) and eliminate trailing .0
        weight = str(round(weight * 2) / 2).replace('.0', '')
        block = ''
        # Amrap rule: 4x4 -> 8 instead of 4/4/4/4/8
        if len(set(reps)) == 2 and len(set(reps[:-1])) == 1:
            block = f'{len(reps)-1}x{reps[0]} -> {reps[-1]}, {weight}lbs'
        
        # When we have 1 rep count for all sets
        elif len(set(reps)) == 1:
            block = f'{len(reps)}x{reps[0]}, {weight}lbs'
        else:
            repStr = '/'.join([str(rep) for rep in reps])
            block = f'{repStr}, {weight}lbs'

        blocks.append(block)
    
    meat = " | ".join(blocks)
    return f'{name}: {meat}'

def generateCaption(s):
    date, exerciseLine = s["Date parsed"], s["Exercise line"]
    # Hardcoded AT EGO for now, no setting for week X day X
    header = f"{date}: AT EGO week X day X"
    caption = "\n".join([header, exerciseLine])
    return caption



In [24]:
df = pd.read_csv("strong.csv", na_filter=False)
df["Date"] = pd.to_datetime(df["Date"])

# To cut short
# df = df.tail(30)
# Group by Date, Exercise and Weight. Can agg reps using .apply. sort=False to preserve exercise order
df = df.groupby(["Date", "Exercise Name", "Weight"], sort=False)["Reps"].agg(list).reset_index()
df = df.groupby(["Date", "Exercise Name"], sort=False).agg(list).reset_index()
df["Exercise Name"] = df["Exercise Name"].str.replace(" \(Barbell\)", "")
df["Exercise Name"] = df["Exercise Name"].str.replace(" \(Dumbbell\)", "")
print(df["Exercise Name"])

# Generate per exercise caption lines
df["Exercise line"] = df.apply(generateLineFromSeries, axis=1)

# Generate per DATE caption lines (joins with newlines)
df = df.groupby(["Date"])["Exercise line"].apply(lambda lines: '\n'.join(lines)).reset_index()
df["Date parsed"] = df["Date"].dt.strftime("-%m/-%d").str.replace("-0", "-").str.replace("-", "")
df["Caption"] = df.apply(generateCaption, axis=1)

# Real ugly but converts date_time to our format m/d. Do this AFTER so dates on different years don't get clumped
# df.tail(2)["Caption"].apply(print)

0                 Overhead Press
1                    Bench Press
2                Cable Crossover
3                  Lateral Raise
4                        Pull Up
                  ...           
1415                   Intensity
1416                    Deadlift
1417    Bench Press - Close Grip
1418          Bicep Curl (Cable)
1419                   Intensity
Name: Exercise Name, Length: 1420, dtype: object


In [180]:
reps = [4, 4, 4, 4, 8]

dog


'abcdef'